In [1]:
import pandas as pd
import torch
import gc
from numba import cuda
from tqdm import tqdm
import json
from datasets import Dataset
import sys
import re
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
import random
import outlines

from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import TextClassificationPipeline
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import pipeline as transformers_pipeline



2025-01-17 11:26:02.422009: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737113162.436485     578 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737113162.440881     578 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-17 11:26:02.458652: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#IMPORT AND PREPARE DATASET

path = '' # Specify the path to your dataset file
ds = pd.read_csv(path)

dataset = Dataset.from_pandas(ds)
print(dataset)


Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'text', 'label', 'target', 'type'],
    num_rows: 3901
})


In [ ]:
hf_token = ""
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
model = outlines.models.transformers("mistralai/Mistral-7B-Instruct-v0.2",device="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
#IMPORT DEFINTIONS - STEP 1
def_step_1 = {  "NO":"",
                "FtW":"Hate is defined as abusive speech targeting specific group characteristics, such as ethnic origin, religion, gender, or sexual orientation.",
                "OL":"Hate Speech is considered any kind of content that conveys malevolent intentions toward a group or an individual.",
                "HSB":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDFoC":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions,  that convey malevolent intentions toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDPC":"Hate speech is considered any kind of content that conveys malevolent intentions such as statements of inferiority, aversion, cursing, calls for exclusion, threaten, harass or violence, and directed toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDT":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDFoC_EDT":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions,  that convey malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDFoC_EDPC":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions, that convey malevolent intentions such as statements of inferiority, aversion, cursing, calls for exclusion, threaten, harass or violence, and directed toward a group or an individual and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDT_EDPC":"Hate speech is considered any kind of content that conveys malevolent intentions such as statements of inferiority, aversion, cursing, calls for exclusion, threaten, harass or violence, toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDFoC_EDPC_EDT":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions,  that conveys malevolent intentions such as statements of inferiority, aversion, cursing, calls for exclusion, threaten, harass or violence, toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members."
                }

def_step_2 = {"HSB_EDFoC_LAA":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions, that convey malevolent intentions toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. ",
             "HSB_EDFoC_LAA_PI":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions, that convey malevolent intentions toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. The outcome of Hate Speech could be the promotion of division among people, undermining of social cohesion in communities, inciting others to commit violence or discrimination, and could have consequences for individuals’ health and safety.",
             "HSB_EDFoC_LAA_Exc":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions, that convey malevolent intentions toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. However, even if it is offensive, it is not considered Hate Speech any content that attacks a person’s personality traits, ideas, or opinions",
             "HSB_EDFoC_LAA_IHS":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions, that convey malevolent intentions toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. Hate Speech can also be implicit, portrayed as an indirect or coded language that uses Irony, Stereotypes, or Misinformation. ",
             "HSB_EDFoC_LAA_PI_Exc":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions, that convey malevolent intentions toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. The outcome of Hate Speech could be the promotion of division among people, undermining of social cohesion in communities, inciting others to commit violence or discrimination, and could have consequences for individuals’ health and safety. However, even if it is offensive, it is not considered Hate Speech any content that attacks a person’s personality traits, ideas, or opinions.",
             "HSB_EDFoC_LAA_Exc_IHS":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions, that convey malevolent intentions toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. Hate Speech can also be implicit, portrayed as an indirect or coded language that uses Irony, Stereotypes, or Misinformation. However, even if it is offensive, it is not considered Hate Speech any content that attacks a person’s personality traits, ideas, or opinions. ",
             "HSB_EDFoC_LAA_PI_IHS":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions, that convey malevolent intentions toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. The outcome of Hate Speech could be the promotion of division among people, undermining of social cohesion in communities, inciting others to commit violence or discrimination, and could have consequences for individuals’ health and safety. Hate Speech can also be implicit, portrayed as an indirect or coded language that uses Irony, Stereotypes, or Misinformation. ",
             "HSB_EDFoC_LAA_PI_IHS_Exc":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions, that convey malevolent intentions toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. The outcome of Hate Speech could be the promotion of division among people, undermining of social cohesion in communities, inciting others to commit violence or discrimination, and could have consequences for individuals’ health and safety. Hate Speech can also be implicit, portrayed as an indirect or coded language that uses Irony, Stereotypes, or Misinformation. However, even if it is offensive, it is not considered Hate Speech any content that attacks a person’s personality traits, ideas, or opinions."
}

def_step_3 = {
                "FtW+":""
}

In [5]:
dataset = dataset.shuffle()

model_inputs = []
labels = []


instructions = "Classify if the following text is considered hate speech or not hate speech:\n"\
              "[TEXT]\n"\
              "Please answer ONLY with 1 if it is hate speech or 0 if it is not hate speech."

for datapoint in tqdm(dataset):
    model_input = instructions
    model_input = model_input.replace("[TEXT]", datapoint['text'])
    
    if datapoint['label'] == 'hate':
        labels.append(1)
    else:
        labels.append(0)
    model_inputs.append(model_input)
print(model_inputs[1])


100%|██████████| 3901/3901 [00:00<00:00, 18730.14it/s]

Classify if the following text is considered hate speech or not hate speech:
pitbulls are one of the most vicious breeds, they should be banned
Please answer ONLY with 1 if it is hate speech or 0 if it is not hate speech.


In [6]:
def metriche(predictions,labels):

    # Calculate precision (handling NaN appropriately)
    precision = precision_score(labels, predictions, average='binary', zero_division=0)

    # Calculate recall (handling NaN appropriately)
    recall = recall_score(labels, predictions, average='binary', zero_division=0)

    # Calculate F1 score (handling NaN appropriately)
    f1 = f1_score(labels, predictions, average='macro', zero_division=0)
    
    return precision, recall, f1
    

In [7]:
def runner(model_inputs, labels, num_runs, id_def, definition, sample):
    results = []

    for j in range(num_runs):  
        if sample == 'all' or sample >= 3901:
            sample = 3901
        else:
            model_inputs = model_inputs[:sample]
            labels = labels[:sample]

        completions = []
        responses = []
        predictions = []
        system_prompt = ''
        model_prompt = "<s>[INST] [SYSTEM_PROMPT] [USER_MESSAGE] [/INST]" #mistral

        
        

        for i, inputtone in enumerate(tqdm(model_inputs)): 
            if id_def == 'NO':
                tot_in = inputtone
            else:
                tot_in = 'Given the following definition of Hate Speech: \n' + str(definition) +'\n' + inputtone

            model_in = model_prompt.replace("[SYSTEM_PROMPT]", system_prompt)
            model_in = model_in.replace("[USER_MESSAGE]", tot_in)

            # Pass the temperature parameter to the generator
            generator = outlines.generate.choice(model, ["1", "0"])
            answer = generator(model_in, max_tokens=3)

            predictions.append(int(answer))

        num = j+1
        print('Run number:', num, 'With definition: ', id_def)     
        numeratore = sum(1 for p,l in zip(predictions,labels) if str(p) == str(l))
        denominatore = len(predictions)
        acc = numeratore/denominatore
        precision, recall, f1 = metriche(predictions, labels)
        print('Accuracy: ',acc)


        precision, recall, f1 = metriche(predictions, labels)

        print('Precision: ', precision)
        print('Recall: ', recall)
        print('F1 Score: ', f1)
        
        results.append({'Run': num,
                        'Model': model_name, 
                        'ID_def': id_def,
                        'Accuracy': acc, 
                        'Precision': precision, 
                        'Recall': recall, 
                        'F1': f1 ,
                        'Input': dataset['text'],
                        'Predictions': predictions, 
                        'Labels': labels, 
                        'Definition': definition,
                        'Target': dataset['target'],
                        'Type': dataset['type']})
                
    return results


In [8]:
#ALL THE DEFINITIONS
sample = 'all'
num_runs = 3
def_id = None
definition = None
step = '+'

results = []

if step == '1':
    def_dict = def_step_1
elif step == '2':
    def_dict = def_step_2
else:
    pass

for k, v in def_step_3.items():
    def_id = k
    definition = v
    result = runner(model_inputs, labels, num_runs=num_runs, id_def=def_id, definition=definition, sample=sample)
    results.extend(result) 

100%|██████████| 3901/3901 [26:05<00:00,  2.49it/s]


Run number: 1 With definition:  FtW+
Accuracy:  0.5452448090233274
Precision:  0.9116279069767442
Recall:  0.3685596088755171
F1 Score:  0.5444098688655998


100%|██████████| 3901/3901 [26:03<00:00,  2.49it/s]


Run number: 2 With definition:  FtW+
Accuracy:  0.5419123301717508
Precision:  0.9067164179104478
Recall:  0.36555095900714557
F1 Score:  0.5410407244079081


100%|██████████| 3901/3901 [26:04<00:00,  2.49it/s]

Run number: 3 With definition:  FtW+
Accuracy:  0.5411432965906178
Precision:  0.9056956115779645
Recall:  0.36479879654005265
F1 Score:  0.5402599061398843


In [9]:
#CREATE DF AND SAVE
df = pd.DataFrame(results)

csv_file_path = '../Outputs/Mistral_FtW_step'+ str(step) +'.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)  # Set index=False to exclude row numbers in the CSV file


In [1]:
import pandas as pd

# Specify the path to your CSV file
csv_file_path = '../Outputs/Mistral_FtW_step1.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Group the DataFrame by 'id_def' and calculate the mean of 'Accuracy' within each group
avg_accuracy_per_id = df.groupby('ID_def')['F1'].mean().reset_index()

avg_accuracy_per_id_sorted = avg_accuracy_per_id.sort_values(by='F1', ascending=False)

# Display the average accuracy for each id_def
print(avg_accuracy_per_id_sorted)


                ID_def        F1
10                  OL  0.572844
9                   NO  0.560452
2            HSB_EDFoC  0.558150
1                  HSB  0.557785
3       HSB_EDFoC_EDPC  0.550438
5        HSB_EDFoC_EDT  0.547773
6             HSB_EDPC  0.547229
7              HSB_EDT  0.541856
0                  FtW  0.538306
4   HSB_EDFoC_EDPC_EDT  0.534558
8         HSB_EDT_EDPC  0.531683


In [3]:
# Group the DataFrame by 'id_def' and calculate the mean of 'Accuracy' within each group
avg_accuracy_per_id = df.groupby('ID_def')['Accuracy'].mean().reset_index()

avg_accuracy_per_id_sorted = avg_accuracy_per_id.sort_values(by='Accuracy', ascending=False)

# Display the average accuracy for each id_def
print(avg_accuracy_per_id_sorted)

                ID_def  Accuracy
10                  OL  0.572930
9                   NO  0.560455
2            HSB_EDFoC  0.558233
1                  HSB  0.557891
3       HSB_EDFoC_EDPC  0.550713
5        HSB_EDFoC_EDT  0.548235
6             HSB_EDPC  0.547552
7              HSB_EDT  0.542425
0                  FtW  0.539263
4   HSB_EDFoC_EDPC_EDT  0.535504
8         HSB_EDT_EDPC  0.532769


In [4]:
# Group the DataFrame by 'id_def' and calculate the mean of 'Accuracy' within each group
avg_accuracy_per_id = df.groupby('ID_def')['Precision'].mean().reset_index()

avg_accuracy_per_id_sorted = avg_accuracy_per_id.sort_values(by='Precision', ascending=False)

# Display the average accuracy for each id_def
print(avg_accuracy_per_id_sorted)

                ID_def  Precision
0                  FtW   0.904788
5        HSB_EDFoC_EDT   0.901256
4   HSB_EDFoC_EDPC_EDT   0.897153
8         HSB_EDT_EDPC   0.896588
7              HSB_EDT   0.895591
3       HSB_EDFoC_EDPC   0.895545
1                  HSB   0.895347
2            HSB_EDFoC   0.893470
6             HSB_EDPC   0.892795
10                  OL   0.883030
9                   NO   0.881917


In [7]:
# Group the DataFrame by 'id_def' and calculate the mean of 'Accuracy' within each group
avg_accuracy_per_id = df.groupby('ID_def')['Recall'].mean().reset_index()

avg_accuracy_per_id_sorted = avg_accuracy_per_id.sort_values(by='Recall', ascending=False)

# Display the average accuracy for each id_def
print(avg_accuracy_per_id_sorted)

                ID_def    Recall
10                  OL  0.430488
9                   NO  0.410054
2            HSB_EDFoC  0.399524
1                  HSB  0.397894
3       HSB_EDFoC_EDPC  0.385859
6             HSB_EDPC  0.382099
5        HSB_EDFoC_EDT  0.378714
7              HSB_EDT  0.372070
0                  FtW  0.362166
4   HSB_EDFoC_EDPC_EDT  0.359784
8         HSB_EDT_EDPC  0.355522
